In [1]:
# The code was removed by Watson Studio for sharing.

# Import Necessary Libraries

In [2]:
import pandas as pd
import numpy as np
import requests
import json
from pandas.io.json import json_normalize

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.17.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00 221.82 kB/s
geopy-1.17.0-p 100% |################################| Time: 0:00:00 338.72 kB/s


/opt/conda/envs/DSX-Python35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/envs/DSX-Python35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/envs/DSX-Python35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/conda/envs/DSX-Python35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.0-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00   1.26 MB/s
branca-0.3.0-p 100% |################################| Time: 0:00:00  31.17 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  31.46 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  39.91 MB/s


In [4]:
toronto_df = pd.read_csv(project.get_file('toronto_neigh_with_latlng.csv'))
toronto_df.head()

,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,North York,Parkwoods,M3A,43.753259,-79.329656
1,North York,Victoria Village,M4A,43.725882,-79.315572
2,Downtown Toronto,"Harbourfront, Regent Park",M5A,43.654260,-79.360636
3,North York,"Lawrence Heights, Lawrence Manor",M6A,43.718518,-79.464763
4,Queen's Park,Queen's Park,M7A,43.664366,-79.392328


In [7]:
# lets get Toronto's location
addr = 'Toronto, ON'
g = Nominatim(user_agent='appjk0132')
loc = g.geocode(addr)
tlat = loc.latitude
tlng = loc.longitude
print(tlat,tlng)

43.653963 -79.387207


# Map with neighborhoods superimposed

In [10]:
map_tor = folium.Map(location=[tlat, tlng], zoom_start=11)

for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

# Foursquare credentials and version

In [11]:
# The code was removed by Watson Studio for sharing.

# Lets get venues in the neighborhoods from foursquare

In [55]:
# limit of number of venues returned by Foursquare API
radius = 500
nearby_venues = []

for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):    
    #build url
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius)
    
    #get json data
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    
    nearby_venues.append([(
        borough,
        neighborhood,
        lat,
        lng,
        v['venue']['name'],
        v['venue']['location']['lat'],
        v['venue']['location']['lng'],
        v['venue']['categories'][0]['name']) for v in venues])

In [56]:
nearby_venues_df = pd.DataFrame([item for nearby_venues in nearby_venues for item in nearby_venues])
nearby_venues_df.columns = ['Borough', 'Neighborhood', 'NeighborhoodLatitude', 'NeighborhoodLongitude', 'Venue', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

In [58]:
nearby_venues_df.groupby('Borough').count()

,Neighborhood,NeighborhoodLatitude,NeighborhoodLongitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
Borough,,,,,,,
Central Toronto,125,125,125,125,125,125,125
Downtown Toronto,500,500,500,500,500,500,500
East Toronto,89,89,89,89,89,89,89
East York,76,76,76,76,76,76,76
Etobicoke,69,69,69,69,69,69,69
Mississauga,11,11,11,11,11,11,11
North York,199,199,199,199,199,199,199
Queen's Park,30,30,30,30,30,30,30
Scarborough,91,91,91,91,91,91,91


In [59]:
project.save_data(data=nearby_venues_df.to_csv(index=False), file_name='toronto_neigh_nearby_venues.csv', overwrite=True)

{'asset_id': 'd163be0d-2f90-4343-871b-9d3091445e24',
 'bucket_name': 'courseracapstoneproject-donotdelete-pr-d28xfsomlxxcfy',
 'file_name': 'toronto_neigh_nearby_venues.csv',
 'message': 'File toronto_neigh_nearby_venues.csv has been written successfully to the associated OS'}